## Deliverable 2. Create a Customer Travel Destinations Map.

In [24]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Saint-Philippe,re,-21.3585,55.7679,78.80,78,20,13.80,light rain
1,1,Algiers,dz,36.7525,3.0420,55.40,82,20,3.44,few clouds
2,2,Batie,bf,9.8833,-2.9167,81.41,19,14,3.56,few clouds
3,3,Port Macquarie,au,-31.4333,152.9167,70.00,94,75,4.61,broken clouds
4,4,Puerto Ayora,ec,-0.7393,-90.3518,77.00,95,10,4.00,clear sky
5,5,Narsaq,gl,60.9167,-46.0500,30.20,86,75,1.14,light snow
6,6,Sao Filipe,cv,14.8961,-24.4956,70.05,76,37,7.61,scattered clouds
7,7,Mataura,pf,-46.1927,168.8643,62.01,67,100,4.00,overcast clouds
8,8,Hithadhoo,mv,-0.6000,73.0833,83.91,76,88,20.24,overcast clouds
9,9,Tuktoyaktuk,ca,69.4541,-133.0374,-11.20,70,40,21.85,snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 120


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Saint-Philippe,re,-21.3585,55.7679,78.80,78,20,13.80,light rain
2,2,Batie,bf,9.8833,-2.9167,81.41,19,14,3.56,few clouds
3,3,Port Macquarie,au,-31.4333,152.9167,70.00,94,75,4.61,broken clouds
4,4,Puerto Ayora,ec,-0.7393,-90.3518,77.00,95,10,4.00,clear sky
6,6,Sao Filipe,cv,14.8961,-24.4956,70.05,76,37,7.61,scattered clouds
7,7,Mataura,pf,-46.1927,168.8643,62.01,67,100,4.00,overcast clouds
8,8,Hithadhoo,mv,-0.6000,73.0833,83.91,76,88,20.24,overcast clouds
10,10,Makakilo City,us,21.3469,-158.0858,82.40,54,20,10.36,few clouds
12,12,Hobart,au,-42.8794,147.3294,79.00,33,0,9.22,clear sky
14,14,Catuday,ph,16.2923,119.8062,84.94,61,14,4.72,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.count()


City_ID                380
City                   380
Country                380
Lat                    380
Lng                    380
Max Temp               380
Humidity               380
Cloudiness             380
Wind Speed             380
Weather description    380
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.count()


City_ID                380
City                   380
Country                380
Lat                    380
Lng                    380
Max Temp               380
Humidity               380
Cloudiness             380
Wind Speed             380
Weather description    380
dtype: int64

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City","City_ID","Country", "Max Temp", "Weather description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,City_ID,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,Saint-Philippe,0,re,78.80,light rain,-21.3585,55.7679,
2,Batie,2,bf,81.41,few clouds,9.8833,-2.9167,
3,Port Macquarie,3,au,70.00,broken clouds,-31.4333,152.9167,
4,Puerto Ayora,4,ec,77.00,clear sky,-0.7393,-90.3518,
6,Sao Filipe,6,cv,70.05,scattered clouds,14.8961,-24.4956,
7,Mataura,7,pf,62.01,overcast clouds,-46.1927,168.8643,
8,Hithadhoo,8,mv,83.91,overcast clouds,-0.6000,73.0833,
10,Makakilo City,10,us,82.40,few clouds,21.3469,-158.0858,
12,Hobart,12,au,79.00,clear sky,-42.8794,147.3294,
14,Catuday,14,ph,84.94,few clouds,16.2923,119.8062,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(index)
    except (IndexError):
        print("Hotel not found... skipping.")
        

0
2
3
4
6
7
8
10
12
14
17
21
23
25
26
27
28
29
30
31
32
34
35
36
37
39
41
46
47
48
50
55
57
58
59
60
63
Hotel not found... skipping.
65
67
68
75
76
77
78
79
80
81
82
83
84
87
89
91
93
Hotel not found... skipping.
97
99
Hotel not found... skipping.
103
104
105
106
108
112
113
114
115
117
118
119
124
125
126
128
129
130
131
Hotel not found... skipping.
137
Hotel not found... skipping.
142
143
146
148
149
150
151
153
157
158
Hotel not found... skipping.
161
164
165
169
170
171
172
176
Hotel not found... skipping.
178
179
181
186
187
191
193
195
197
198
205
207
208
210
212
216
218
219
220
221
222
224
227
228
229
236
237
Hotel not found... skipping.
244
245
Hotel not found... skipping.
249
250
251
254
255
256
257
258
260
263
266
267
268
Hotel not found... skipping.
272
275
276
278
279
281
283
284
285
286
287
290
291
293
295
297
299
300
302
303
305
308
309
311
312
314
318
320
321
322
324
327
329
332
Hotel not found... skipping.
335
336
338
339
340
342
343
344
Hotel not found... skipping.
346

In [31]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df = hotel_df.dropna()

hotel_df

,City,City_ID,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,Saint-Philippe,0,re,78.80,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
2,Batie,2,bf,81.41,few clouds,9.8833,-2.9167,Auberge SONDAR AFACI
3,Port Macquarie,3,au,70.00,broken clouds,-31.4333,152.9167,Rydges Port Macquarie
4,Puerto Ayora,4,ec,77.00,clear sky,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Sao Filipe,6,cv,70.05,scattered clouds,14.8961,-24.4956,Tortuga B&B
...,...,...,...,...,...,...,...,...
675,Santarem,675,br,78.80,scattered clouds,-2.4431,-54.7083,Ecotur Lodge
677,Dunedin,677,nz,61.00,scattered clouds,-45.8742,170.5036,Scenic Hotel Southern Cross
678,Pathein,678,mm,89.46,scattered clouds,16.7833,94.7333,New Pammawaddy Hotel
680,Eyl,680,so,72.91,broken clouds,7.9803,49.8164,Amiin Hotel


In [32]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [35]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Weather description</dt><dd>{Weather description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))